<a href="https://www.kaggle.com/code/nguyenvanquanghust/cifartransferlearning?scriptVersionId=90429788" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
from keras.models import Model
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(xtrain, ytrain),(xtest, ytest) = cifar10.load_data()

In [ ]:
xtrain.shape

In [ ]:
xtrain = xtrain/255.0
xtest = xtest/255.0

In [ ]:
ytrain = np_utils.to_categorical(ytrain,10)

In [ ]:
ytrain

In [ ]:
trainGenerator = ImageDataGenerator(rotation_range=2,horizontal_flip=True, zoom_range=0.1)
testGenerator = ImageDataGenerator(rotation_range=2,horizontal_flip=True, zoom_range=0.1)

In [ ]:
baseModel = VGG16(include_top=False, weights='imagenet', input_shape=(32,32,3))

In [ ]:
baseModel.summary()

In [ ]:
baseOut = baseModel.output
baseOut = Flatten(name='flatten')(baseOut)
baseOut = Dense(1024, activation='relu')(baseOut)
baseOut = Dense(512, activation='relu')(baseOut)
baseOut = Dense(256, activation='relu')(baseOut)
baseOut = Dense(128, activation='relu')(baseOut)
baseOut = Dense(10, activation='softmax')(baseOut)

In [ ]:
fullModel = Model(baseModel.input, baseOut)

In [ ]:
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
fullModel.summary()

In [ ]:
sgd = SGD(0.001)
fullModel.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fullModel.fit_generator(trainGenerator.flow(xtrain, ytrain, batch_size=100), epochs=20, steps_per_epoch=xtrain.shape[0]//100, validation_data= testGenerator.flow(xtest, ytest, batch_size=100), verbose=1 )

In [ ]:
for layer in baseModel.layers[14:]:
    layer.trainable = True

In [ ]:
sgd = SGD(0.001)
fullModel.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fullModel.fit_generator(trainGenerator.flow(xtrain, ytrain, batch_size=100), epochs=20, steps_per_epoch=xtrain.shape[0]//100, validation_data= testGenerator.flow(xtest, ytest, batch_size=100), verbose=1 )

In [ ]:
fullModel.fit_generator(trainGenerator.flow(xtrain, ytrain, batch_size=100), epochs=50, steps_per_epoch=xtrain.shape[0]//100, validation_data= testGenerator.flow(xtest, ytest, batch_size=100), verbose=1 )

In [ ]:
labelName = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


In [ ]:
for i in range(20):
  predict_test = fullModel.predict(xtest[i].reshape(1,32,32,3))
  plt.figure(figsize=(7,2))
  plt.imshow(xtest[i].reshape(32,32,3))
  plt.xlabel(labelName[np.argmax(predict_test)])